# Clasificación de datos posterior al agrupamiento

Utilizar un arbol de decisión para determinar las variables más importantes para el clustering y la precisión del modelo

In [1]:
import os
import sys
%pip install -q ipywidgets
import ipywidgets as widgets

directorio = '../Results'
# directorio = '/content'

# Leer archivos xlsx dentro de la carpeta Data
files = os.listdir(directorio)
files = [f for f in files if f.endswith('.xlsx')]

# Seleccionar archivo a leer
dropdown = widgets.Dropdown(
  options=[('Seleccionar archivo', None)] + [(f, f) for f in files],
  description='Archivo:',
  disabled=False,
)
nombre_archivo = '';
def on_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    if change['new'] is not None:
      global nombre_archivo
      nombre_archivo = change['new']
      print('Archivo seleccionado:', nombre_archivo)

dropdown.observe(on_change)
display(dropdown)

Note: you may need to restart the kernel to use updated packages.


Dropdown(description='Archivo:', options=(('Seleccionar archivo', None), ('02_clursters_df_previo_imputacion_n…

In [ ]:


ruta = os.path.join(directorio, nombre_archivo)

# Data split y seleccion de modelos
dataset = pd.read_excel(ruta)

X = dataset.drop(columns=['cluster_kmeans', 'cluster_hc', 'Comunidad'])
y = dataset['cluster_kmeans']
display(X.head())
display(y.head())


In [ ]:
# Ver balance de clases
print(y.value_counts())
# histograma de clases

import matplotlib.pyplot as plt
plt.hist(y)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Identificar variables categóricas y numéricas
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [ ]:
# from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
# importar pipeline, column transformer y gridsearch
# from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
#Preprocesamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer ya no es necesario por que l
from sklearn.tree import DecisionTreeClassifier



# Crear un pipeline para preprocesar los datos
numerical_transformer = Pipeline(steps=[
  ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar los preprocesadores
preprocessor = ColumnTransformer(
  transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
  ])

pipeline = Pipeline(steps=[
  ('preprocessor', preprocessor),
  ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
  ('classifier', DecisionTreeClassifier())
])

# Definir los hiperparámetros a buscar
param_grid = {
  'classifier__max_depth': [None, 5, 10, 15, 20],
  'classifier__min_samples_split': [2, 5, 10]
}

# Definir el esquema de validación cruzada (StratifiedKFold)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Realizar la búsqueda en cuadrícula con validación cruzada en el pipeline
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy')

# Ajustar los datos usando GridSearchCV
grid_search.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Ajustar el mejor modelo en los datos de entrenamiento
best_model.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
best_params = grid_search.best_params_
print("Best Hyperparameters:\n", best_params, '\n')

# Predecir los datos de prueba
y_pred = best_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy, '\n')

# Calcular la precisión
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision:", precision, '\n')

# Calcular el reporte de clasificación
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=best_model.classes_)
disp.plot()


In [ ]:
from sklearn.model_selection import cross_val_score
# usando los mejores hiperparámetros
# Best Hyperparameters: {'classifier__max_depth': 10, 'classifier__min_samples_split': 5}
clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5)

stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

preprocessor = ColumnTransformer(
  transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
  ])

pipeline = Pipeline(steps=[
  ('preprocessor', preprocessor),
  ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
  ('classifier', clf)
])

# Calcular la precisión usando validación cruzada
scores = cross_val_score(pipeline, X, y, cv=stratified_kfold, scoring='accuracy')

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Matriz de confusión
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# X_train, X_test, y_train, y_test 
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=pipeline.classes_)
disp.plot()

plt.show()
    

In [ ]:

# Evalauamos en el train set solo para ver que no quedó sobreajustado
train_accuracy = best_model.score(X_train, y_train)
print("Train Accuracy:", train_accuracy)

# Evaluamos en el test set para ver como generaliza
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Obtenemos las variables más importantes
importances = best_model.named_steps['classifier'].feature_importances_

# Ordenamos las importancias de mayor a menor
indices = np.argsort(importances)[::-1]

# Obtenemos los nombres de las columnas
columnas = X_train.columns
display(columnas)
# Mostramos las 10 variables más importantes
for i in range(5):
    print(f"{columnas[indices[i]]}: {importances[indices[i]]}")